In [ ]:
import functools
import os
import geopandas as gpd
import json

from pathlib import Path
from traitlets import (
    Any, link, observe, Unicode, HasTraits
)
import ipyvuetify as v
import sepal_ui.sepalwidgets as sw

In [ ]:
from sepal_ui.scripts import utils as su
from sepal_ui.mapping import mapping as m

In [ ]:
def alert_error(alert):
    """Decorator to execute try/except sentence
    and toggle loading button object
    
    Params:
        alert (sw.Alert): Alert to display errors
    """
    def decorator_alert_error(func):
        @functools.wraps(func)
        def wrapper_alert_error(*args, **kwargs):
            try:
                value = func(*args, **kwargs)
            except Exception as e:
                alert.add_msg(f'{e}', type_='error')
                raise e
            return value
        return wrapper_alert_error
    return decorator_alert_error


In [ ]:
class Flex(v.Flex, sw.SepalWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [ ]:
class AoiView(v.Card):
    
    method = Unicode('').tag(sync=True)
    column = Any('').tag(sync=True)
    field = Any('').tag(sync=True)
    
    def __init__(self, model, map_, *args, **kwargs):
        
        self.map_=map_
        self.methods = self._get_methods()
        
        self.alert = sw.Alert()
        self.model = model(self.alert)
        
        w_method = v.Select(
            label = 'Select a method',
            v_model = self.method,
            items = self._get_methods()
        )
        
        
        self.w_countries = v.Select(
            label="Select country",
            v_model='',
            items=self._get_countries(),
        )
        self.btn_country = sw.Btn('Select', small=True)
        w_countries_btn = Flex(
            class_='d-flex align-center mb-2',
            row=True, 
            children=[self.w_countries, self.btn_country])
        
        
        self.w_file = sw.FileInput(['.shp'], os.getcwd())
        self.btn_file = sw.Btn('Select', small=True)
        w_file_btn = Flex(
            class_='d-flex align-center mb-2',
            row=True, 
            children=[self.w_file, self.btn_file])
        
        self.w_column = v.Select(
            _metadata={'name':'column'},
            label="Filter by column",
            v_model=self.column,
            items=[]
        )
        
        self.w_field = v.Select(
            _metadata={'name':'field'},
            label="Select field",
            v_model=self.field,
            items=[]
        )
        
        self.components = {
            'Country' : w_countries_btn,
            'Upload file' : w_file_btn,
            'Column' : self.w_column,
            'Field' : self.w_field
        }
        
        self._hide_components()
        
        super().__init__(*args, **kwargs)
        
        
        # Link traits view
        link((self, 'method'),(w_method, 'v_model'))
        link((self, 'column'),(self.w_column, 'v_model'))
        link((self, 'field'),(self.w_field, 'v_model'))
        
        # Link traits with model
        link((self.model, 'country'),(self.w_countries, 'v_model'))
        link((self.model, 'file'),(self.w_file, 'file'))
        
        # Events
        self.btn_file.on_event('click', self._file_btn_event)
        
        self.children=[
            self.alert,
            w_method,
            w_countries_btn,
            w_file_btn,
        ]
            
    def _file_btn_event(self, widget, event, data):
        """Define behavior when the file button is clicked"""
        
        # Create a geopandas dataset
        self.model.shape_to_gpd()
        
        # Populate columns widget with all columsn plus 'ALL' in case user wants to use all 
        self.w_column.items = ['_ALL'] + self.model._get_columns()
        
        # Trigger field, and column widget, so user can select them
        self._add_widget(self.w_column, 'column')
    
    @observe('column')
    def _get_fields(self, change):
        """Populate widget field items with fields"""
        
        # Reset fields items
        self.w_field.items = []
        
        if self.column == '_ALL':
            "All geometries were selected"
        else:
            self.w_field.items = self.model._get_fields(self.column)
            self._add_widget(self.w_field, 'field')
    
    @observe('field')
    def _get_selected_feature(self, change):
        """Define selected feature with the current options"""

        self.model.selected_feature = self.model._get_selected(self.column, self.field)
            
        
    def _add_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        # Almost all datasets are composed by more than one feature(geometry)
        # this function will allow only select one (or more?) geometries.
        su.show_component(widget)
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name not in widgets_name:
            # If not already 'filter' widgets add them to the view
            self.children = self.children + [widget]

    def _hide_components(self):
        """Hide all possible componentes"""
        
        for component in self.components.values():
            su.hide_component(component)
        
    def _get_methods(self):
        """Handle which methods will be displayed in select widget"""
        
        return ['Draw on map', 'Country', 'Upload file']
    
    def _get_countries(self):
        """Create a list of countries"""
        
        return list(range(10))
    
    @observe('method')
    def _aoi_method_event(self, change):
        
        method = change['new']
        self._hide_components()
        
        if method == 'Draw on map':
            self.map_.show_dc()
        else:
            self.map_.hide_dc()
            su.show_component(self.components[method])

In [ ]:
class AoiModel(HasTraits):
    
    country = Any('').tag(sync=True)
    file = Unicode('').tag(sync=True)

    
    def __init__(self, alert, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.alert = alert
        self.gpd = None
        self.selected_feature = None
        
        # Decorate methods
        self.shape_to_gpd = alert_error(self.alert)(self.shape_to_gpd)
            
    def shape_to_gpd(self):
        """ Converts shapefile into geopandas"""
        
        file_path = Path(self.file)
        
        assert file_path.exists(), "File doesn't exists"
        
        if file_path.suffix == '.shp':

            self.gdf = gpd.read_file(str(file_path))
            self.gdf = self.gdf.to_crs("EPSG:4326")
            
    def _get_columns(self):
        """Return all columns skiping geometry"""
        return list(set(['geometry'])^set(self.gdf.columns.to_list()))
        
    def _get_fields(self, column):
        """Return fields from selected column"""
        return self.gdf[column].to_list()
    
    def _get_selected(self, column, field):
        """Get selected element"""
        
        return self.gdf[self.gdf[column] == field]
        

In [ ]:
map_=m.SepalMap()

In [ ]:
aoi = AoiView(map_=map_, model = AoiModel)

In [ ]:
aoi

In [ ]:
aoi.column

In [ ]:
aoi.field

In [ ]:
aoi.model.file

In [ ]:
aoi.model.selected_feature